In [ ]:
import sys
import os
import findspark
findspark.init()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
import spark
import pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [ ]:
user = os.getenv("user", default=None)
password = os.getenv("password", default=None)

In [ ]:
#creating a spark session
spark = SparkSession.builder.appName('capstone_project').getOrCreate()

In [ ]:
cdw_sapp_branch = os.path.join("files", "cdw_sapp_branch.json")
df_branch = spark.read.json(cdw_sapp_branch)

In [ ]:
type(df_branch)
df_branch.show()

In [ ]:
df_branch.printSchema()

In [ ]:
df_branch.createOrReplaceTempView('branch_data')

In [ ]:
spark.sql("SELECT BRANCH_CODE, BRANCH_NAME, BRANCH_STREET, BRANCH_CITY, BRANCH_STATE, \
IFNULL(branch_zip, 00000) BRANCH_ZIP, CONCAT('(', SUBSTRING(branch_phone,1,3),')', SUBSTRING(branch_phone, 4,3), '-', SUBSTRING(branch_phone,7)) BRANCH_PHONE, LAST_UPDATED \
FROM branch_data")

In [ ]:
branch_df = spark.sql("SELECT CAST(BRANCH_CODE AS INT) BRANCH_CODE, BRANCH_NAME, BRANCH_STREET, BRANCH_CITY, BRANCH_STATE, \
IFNULL(branch_zip, 00000) BRANCH_ZIP, CONCAT('(', SUBSTRING(branch_phone,1,3),')', SUBSTRING(branch_phone, 4,3), '-', SUBSTRING(branch_phone,7)) BRANCH_PHONE, LAST_UPDATED \
FROM branch_data")
branch_df.show()

In [ ]:
branch_df.printSchema()

In [ ]:
branch_df.createOrReplaceTempView('branch_df')

In [ ]:
branch_df = spark.sql("SELECT BRANCH_CODE, BRANCH_NAME, BRANCH_STREET, BRANCH_CITY, BRANCH_STATE, \
CAST(BRANCH_ZIP as INT) BRANCH_ZIP,BRANCH_PHONE , LAST_UPDATED \
FROM branch_df")

In [ ]:
branch_df.printSchema()

In [ ]:
branch_df.show()

In [ ]:
df_branch = branch_df.withColumn('LAST_UPDATED', F.to_timestamp('LAST_UPDATED'))

In [ ]:
df_branch.printSchema()

In [ ]:
df_branch.show()

In [ ]:
df_branch.printSchema()

In [ ]:
df_branch.show()

In [ ]:
# 'DROP TABLE `creditcard_capstone`.`cdw_sapp_branch`'

In [ ]:
df_branch.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.cdw_sapp_branch") \
.option("user", user) \
.option("password", password) \
.save()

In [ ]:
spark.stop()

In [ ]:
# ALTER TABLE `creditcard_capstone`.`cdw_sapp_branch` 
# CHANGE COLUMN `BRANCH_NAME` `BRANCH_NAME` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `BRANCH_STREET` `BRANCH_STREET` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `BRANCH_CITY` `BRANCH_CITY` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `BRANCH_STATE` `BRANCH_STATE` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `BRANCH_PHONE` `BRANCH_PHONE` VARCHAR(45) NULL DEFAULT NULL ;

In [ ]:
# ALTER TABLE `creditcard_capstone`.`cdw_sapp_branch` 
# CHANGE COLUMN `BRANCH_CODE` `BRANCH_CODE` INT(11) NOT NULL ,
# ADD PRIMARY KEY (`BRANCH_CODE`);
